<h1 align="center">Segmenting and Clustering Neighborhoods in Toronto</h1> 
<h3 align="center">by Thithat Promlikitchai</h3> 
<h4 align="center">IBM Applied Data Science Capstone</h4> 

<h3 align="center"><em>Part One: Webscraping Wikipedia Page and Creating the Postal Code-Borough-Neighborhood Dataframe</em></h3> 

In [231]:
## Importing needed dependencies

import pandas as pd
pd.set_option('display.max.rows',None,'display.max.colwidth',None)
from bs4 import BeautifulSoup 
import requests

print('Libraries installed!')

Libraries installed!


In [232]:
## Retrieving URL and creating BeautifulSoup object

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data = requests.get(url).text
soup = BeautifulSoup(data,"html5lib")
table = soup.find('table')

In [233]:
## Scraping through BeautifulSoup object

table_contents = []
for row in table.find_all('td'): 
    cell = {}
    if row.span.text == 'Not assigned':
        pass
    else:
        cell['Postal Code'] = row.p.text[0:3]
        cell['Borough'] = row.span.text.split('(')[0]
        cell['Neighborhood'] = row.span.text.split('(')[1].strip(')').replace(' /',',').replace(')',' ').strip(' ')
        table_contents.append(cell)

In [234]:
## Converting list into dataframe and cleaning data

toronto = pd.DataFrame(table_contents)
toronto['Borough']=toronto['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
toronto.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [235]:
## Checking dataframe

# If all true, there are no null values in the dataframe
print ((toronto['Postal Code'].isnull().sum()) == 0)
print ((toronto['Borough'].isnull().sum()) == 0)
print ((toronto['Neighborhood'].isnull().sum()) == 0)
print('\n')
#If all true, there are no 'Not assigned'
print(((toronto['Borough'] == 'Not assigned').sum())== 0)
print(((toronto['Neighborhood'] == 'Not assigned').sum())== 0)

True
True
True


True
True


In [236]:
## Number of rows in dataframe is equal to 103 FSAs

print ('There are {} rows in the dataframe'.format(toronto.shape[0]))

There are 103 rows in the dataframe


<h3 align="center"><em>Part Two: Geocoding Dataframe with Longitudes and Latitudes</em></h3> 

In [237]:
## Importing needed dependencies

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim

!pip install folium
import folium

print('Libraries installed!')

Solving environment: done

# All requested packages already installed.

Libraries installed!


In [240]:
## Creating a function to retrieve coordinates

def getcoordinates(postal_code):
    geolocator = Nominatim(user_agent="toronto_explorer")
    location = geolocator.geocode('{},Toronto, Ontario'.format(postal_code))
    if location == None:
        pass
    else:
        latitude = location.latitude
        longitude = location.longitude
        return latitude,longitude

In [241]:
## Testing out function

postalcode_list = list(toronto['Postal Code'])
coordinates_list = []

for code in postalcode_list:
    result = getcoordinates(code)
    coordinates_list.append(result)

print(len(coordinates_list) - coordinates_list.count(None))

21


#### __The Nominatim function returned only 21 coordinates out of 103 postal codes, so the alternative is to retrieve all the coordinates from the provided csv file.__

In [242]:
## Reading csv file into dataframe

url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv'
geospatial = pd.read_csv(url)
geospatial.shape

(103, 3)

In [243]:
## Merging the two dataframes to obtain a new dataframe of geocoded postal codes

coded_toronto = pd.merge(toronto,geospatial, how = 'inner', on = 'Postal Code')

print(coded_toronto.shape)
print(coded_toronto['Latitude'].isnull().sum(), 'null latitude values')
print(coded_toronto['Longitude'].isnull().sum(), 'null longitude values')

(103, 5)
0 null latitude values
0 null longitude values


In [244]:
coded_toronto.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [247]:
## Visualizing all postal codes in Toronto

address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude


map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11, tiles = 'Stamen Toner')

# add markers to map
for lat, lng, code, borough in zip(coded_toronto['Latitude'], coded_toronto['Longitude'], coded_toronto['Postal Code'], coded_toronto['Borough']):
    label = '{}, {}'.format(code, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='orange',
        fill=True,
        fill_color='orange',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<h3 align="center"><em>Part Three: Clustering Toronto Neighborhoods using Foursquare and KMeans</em></h3> 

In [443]:
## Importing needed dependencies

from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import json
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
from IPython.display import clear_output

print('Libraries installed!')

Libraries installed!


In [455]:
## Defining Foursquare credentials

CLIENT_ID = 'AT3IP12B2N0GAD3O4AA200X4IPEY14MSOI4EUNHSHH2WYMCQ' 
CLIENT_SECRET = 'I4QX32UGBRPVT3BMLD5UG5FVKV5BM2JOPGIYID2SSDFYGLKW' 
ACCESS_TOKEN = 'URA4WHC5EPR3T1XJIZ42RUT3Y4K5HL51BYYD3QUSQFQZB225' 
VERSION = '20180605'
LIMIT = 100

print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

CLIENT_ID: AT3IP12B2N0GAD3O4AA200X4IPEY14MSOI4EUNHSHH2WYMCQ
CLIENT_SECRET:I4QX32UGBRPVT3BMLD5UG5FVKV5BM2JOPGIYID2SSDFYGLKW


#### __800 meters was used as the default search radius for the Foursquare API. This measure is based on the walking catchment defined by the principle of 20-minute neighborhoods in urban planning.__

In [456]:
## Creating functions to retrieve nearby venues and extract the category of the venues
    
def getnearbyvenues(postal_code, latitudes, longitudes, radius=800):
    
    venues_list=[]
    for code, lat, long in zip(postal_code, latitudes, longitudes):
        print(code)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            long, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        venues_list.append([(
            code, 
            lat, 
            long, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postal Code', 
                  'Postal Code Latitude', 
                  'Postal Code Longitude', 
                  'Venue Name', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [457]:
## Creating a new dataframe with venues for each postal code

toronto_venues = getnearbyvenues(coded_toronto['Postal Code'], coded_toronto['Latitude'], coded_toronto['Longitude'])

M3A
M4A
M5A
M6A
M7A
M9A
M1B
M3B
M4B
M5B
M6B
M9B
M1C
M3C
M4C
M5C
M6C
M9C
M1E
M4E
M5E
M6E
M1G
M4G
M5G
M6G
M1H
M2H
M3H
M4H
M5H
M6H
M1J
M2J
M3J
M4J
M5J
M6J
M1K
M2K
M3K
M4K
M5K
M6K
M1L
M2L
M3L
M4L
M5L
M6L
M9L
M1M
M2M
M3M
M4M
M5M
M6M
M9M
M1N
M2N
M3N
M4N
M5N
M6N
M9N
M1P
M2P
M4P
M5P
M6P
M9P
M1R
M2R
M4R
M5R
M6R
M7R
M9R
M1S
M4S
M5S
M6S
M1T
M4T
M5T
M1V
M4V
M5V
M8V
M9V
M1W
M4W
M5W
M8W
M9W
M1X
M4X
M5X
M8X
M4Y
M7Y
M8Y
M8Z


In [354]:
## Dropping duplicated venues in the dataframe

toronto_venues.drop_duplicates(subset = 'Venue Name', keep = 'first', inplace = True);

In [356]:
## Grouping the new dataframe by postal code to examine the number of venues returned for each code

toronto_venues.groupby('Postal Code').count()[['Venue Name']]

,Venue Name
Postal Code,
M1B,9
M1C,5
M1E,7
M1G,5
M1H,18
M1J,4
M1K,8
M1L,10
M1M,3


In [357]:
## Finding the total number of unique venue categories

toronto_venues['Venue Category'].nunique()

334

#### __By exploring the venues in each postal code in Toronto, there are 334 unique venue categories. Several duplicated venues were dropped, suggesting that some postal codes are located close enough to each other to result in an overlapping search radius when using the Foursquare API. Furthermore, venues were returned for only 100 postal codes out of the total of 103.__

In [366]:
## One-hot encoding the venue dataframe

toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix = "", prefix_sep = "")
toronto_PC = toronto_venues[['Postal Code']]

toronto_onehotPC = pd.merge(toronto_PC,toronto_onehot,left_index = True, right_index = True)
toronto_onehotPC.head()

,Postal Code,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Peking Duck Restaurant,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Lab,Pier,Pilates Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping P

In [367]:
## Grouping the dataframe by postal code and calculating the frequency of occurence for each venue category

toronto_grouped = toronto_onehotPC.groupby('Postal Code').mean().reset_index()
toronto_grouped.head()

,Postal Code,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Peking Duck Restaurant,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Lab,Pier,Pilates Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping P

In [416]:
## Using the above dataframe to create a new dataframe with the 10 most common venue categories for each postal code

def most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10
indicators = ['st', 'nd', 'rd']

columns = ['Postal Code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

postalcode_venues = pd.DataFrame(columns=columns)
postalcode_venues['Postal Code'] = toronto_grouped['Postal Code']

for ind in np.arange(toronto_grouped.shape[0]):
    postalcode_venues.iloc[ind, 1:] = most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

postalcode_venues.head()

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Trail,Paper / Office Supplies Store,African Restaurant,Chinese Restaurant,Spa,Fast Food Restaurant,Coffee Shop,Bus Station,New American Restaurant,Organic Grocery
1,M1C,Breakfast Spot,Bar,Burger Joint,Italian Restaurant,Accessories Store,Neighborhood,Organic Grocery,Optical Shop,Office,Noodle House
2,M1E,Laundromat,Pizza Place,Supermarket,Rental Car Location,Restaurant,Fried Chicken Joint,Greek Restaurant,Accessories Store,Music Venue,Noodle House
3,M1G,Park,Construction & Landscaping,Convenience Store,Business Service,Accessories Store,Nail Salon,Optical Shop,Office,Noodle House,Nightclub
4,M1H,Indian Restaurant,Yoga Studio,Hakka Restaurant,Burger Joint,Music Store,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,Chinese Restaurant,Bakery


In [417]:
## Segmenting the postal codes into 3 clusters using KMeans and creating the final dataframe

kclusters = 3
toronto_clustering = toronto_grouped.drop('Postal Code', axis = 1)
kmeans = KMeans(n_clusters = kclusters, random_state = 42).fit(toronto_clustering)

postalcode_venues.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_final = coded_toronto
toronto_final = toronto_final.join(postalcode_venues.set_index('Postal Code'), on='Postal Code')
toronto_final.dropna(axis = 0, inplace = True)
toronto_final['Cluster Labels'] = toronto_final['Cluster Labels'].astype(int)

In [418]:
toronto_final.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,2,Bus Stop,Park,Food & Drink Shop,Intersection,Accessories Store,Nail Salon,Office,Noodle House,Nightclub,New American Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,2,Portuguese Restaurant,Hockey Arena,Sporting Goods Shop,Park,Coffee Shop,French Restaurant,Nail Salon,Optical Shop,Office,Noodle House
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Park,Italian Restaurant,Bakery,Theater,Café,Pub,Restaurant,Sushi Restaurant,Breakfast Spot
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1,Furniture / Home Store,Dessert Shop,Rental Car Location,Restaurant,Clothing Store,Athletics & Sports,Vietnamese Restaurant,Accessories Store,Boutique,Gift Shop
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,1,Coffee Shop,Japanese Restaurant,Park,Bubble Tea Shop,Sushi Restaurant,French Restaurant,Gastropub,Café,Sandwich Place,Office


In [439]:
## Visualizing clusters in Toronto created using postal codes

address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11, tiles = 'Stamen Toner')

markers_colors = {'0':'#14e8ff','1':'#3b838c','2':'#6af7c6'}
for lat, long, code, cluster in zip(toronto_final['Latitude'], toronto_final['Longitude'], toronto_final['Postal Code'], toronto_final['Cluster Labels']):
    label = folium.Popup(str(code) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color= 'black',
        weight = 0.5,
        fill=True,
        fill_color=markers_colors[str(cluster)],
        fill_opacity=1).add_to(map_clusters)
       
map_clusters

#### __Despite dropping duplicated venues prior to running KMeans, the result of segmenting postal codes in Toronto into clusters didn't produce sufficient variation. It is unlikely that Toronto neighborhoods are homogenous across the board. The observed clustering is more likely due to limitations caused by the data obtained, which could be addressed by adjusting the search radius for the Foursquare API or setting a higher return limit.__